# NER CRF - Atos de Contratos

In [1]:
import pandas as pd
import nltk
import scipy.stats
import sklearn
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
#import matplotlib.pyplot as plt
#%matplotlib inline
nltk.download('punkt')
import numpy as np

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

[nltk_data] Downloading package punkt to
[nltk_data]     /home/lucelia_vieira/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/home/lucelia_vieira/experiments/members/lucelia/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#configuraçao para que as variáveis não mudem quando executadas nas mesmas condições pelo modelo
import random
import torch
seed = 3
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
!export CUBLAS_WORKSPACE_CONFIG=:16:8

## Converte CSV para Padrão IOB

In [3]:
train_path = './CSVs/ContratosLicitacoes/extrato_contrato_train.conll'
train = open(train_path, 'r').readlines()

test_path = './CSVs/ContratosLicitacoes/extrato_contrato_train.conll'
test = open(test_path, 'r').readlines()

In [ ]:
train

In [4]:
#Cria uma lista dentro de outra para executar a extração das featuress
def preprocess(sentences):
    l_sentences = []
    l1_ = []
    for token in sentences[1:]: 
        #print('token==>', token)
        cls = token.split()    
        #print('token.split==>', cls)
        if len(cls) != 0:
            l1_.append(cls)
            #print('apos o append==>', l1_)
        else:
            l_sentences.append(l1_)
            l1_ = []
    return l_sentences

In [5]:
#Quebra a sentença em lista
train_1=preprocess(train)

In [ ]:
#Cria uma Lista 
# train_list = []
# for i in train_1:
#    if not isinstance(i, list):
#       train_list.append(i)
#    else:
#       for j in i:
#         train_list.append(j)

# len(train_list)

In [6]:
for item in train:
    #print(item)
    if (len(item)>4):
     #   print("ok")

SyntaxError: unexpected EOF while parsing (909956513.py, line 4)

## Create dictionary feature for each word in each sequence in x

In [7]:
def word2features(sent, i):
    
    word = sent[i][0]
    #print ('word', word)
    #postag = sent[i][1]
    #print ('postag', postag)

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        #'postag': postag,
        #'postag[:2]': postag[:2],        
    }
    if i > 0:
        word1 = sent[i-1][0]
        #postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            #'-1:postag': postag1,
            #'-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        #postag1 = sent[i+1][1]        
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            #'+1:postag': postag1,
            #'+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
                
    return features

In [8]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, __, label in sent]

def sent2tokens(sent):
    return [token for token, postag, __, label in sent]

In [9]:
X_train = [sent2features(s) for s in train_1]

y_train = [sent2labels(s) for s in train_1]

In [ ]:
len(X_train)

In [10]:
X_train

Listando as tags

In [ ]:
tags = set()

for label in y_train:
    for tag in label:
        tags.add(tag)
tags = list(tags)
num_tags = len(tags)

In [ ]:
tags

In [ ]:
test_1=preprocess(test)

In [ ]:
x_test_word=[x[0] for x in test_1[0]]
y_test_label=[x[-1] for x in test_1[0]]

In [ ]:
test_sentence = [list(zip(x_test_word,y_test_label))]

In [ ]:
def sent2featuresTest(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labelsTest(sent):
    return [label for token, label in sent]

def sent2tokensTest(sent):
    return [token for token, label in sent]

In [ ]:
X_test = [sent2featuresTest(s) for s in test_sentence]

y_test = [sent2labelsTest(s) for s in test_sentence]


## Model CRF Trainning

In [ ]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics


crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    #c1=10,
    #c2=0.1,
    c1=0.5, 
    c2=1e-3, 
    max_iterations=100,
    #max_iterations=50,
    
    all_possible_transitions=False,
    all_possible_states=True
)

#crf.fit(x_train, y_train)

try:
    #crf.fit(x_train[:200], y_train[:200])
    crf.fit(X_train, y_train)
except AttributeError:
    pass


## Evaluation

In [ ]:
classes = list(crf.classes_)
classes.remove('O')

y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=classes)

In [ ]:
#crf.classes_

In [ ]:
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2
from seqeval.metrics import f1_score
from seqeval.scheme import IOB1
from seqeval.scheme import IOBES

#classification_report(labels_skweak, gold_y_test,  mode='strict', scheme=IOB2)
#
#for item in classes:
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2
from seqeval.metrics import f1_score

report = classification_report(y_test, y_pred, output_dict=False, mode='strict', scheme=IOB2)
print(report)
#np.save("./Results/METRICA_EXTRATO_CONTRATO_V1_0_sq.npy", report)

## Hyperparameter Optimization

In [ ]:
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True,
    all_possible_states=True
)
params_space = {
    'c1': scipy.stats.expon(scale=15.0),
    'c2': scipy.stats.expon(scale=1.0),
}


# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=classes)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(x_train_s, y_train_s)

In [ ]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

## Check best estimator on our test data

In [ ]:
sorted_classes = sorted(
    classes,
    key=lambda name: (name[1:], name[0])
)

In [ ]:
crf = rs.best_estimator_
y_pred = crf.predict(x_test_s)
print(metrics.flat_classification_report(
    y_test_s, y_pred, labels=sorted_classes, digits=3
))

## Let’s check what classifier learned

In [ ]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

## Saving the best model

In [ ]:
import joblib

joblib.dump(crf, 'crf_model.pkl')

In [ ]:
model = joblib.load('.pklcrf_model')

In [ ]:
model

In [ ]:
model.classes_